In [ ]:
import os, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.chrome.options import Options

DOWNLOAD_DIR = r"C:\Users\USER\Downloads"
options = Options()
options.add_experimental_option("detach", True)
options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
})

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            for f in new_files:
                if (f.endswith(".xlsx") or f.endswith(".xls")) and not f.startswith("~"):
                    return f
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0 
    while retries < 2: 
        try:
            time.sleep(0.5) ### 이 sleep은 팝업창 로딩 대기. WebDriverWait으로 팝업 내 요소가 나타나는 것을 기다리는 것으로 대체 시 제거 가능. 
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()
            time.sleep(4) ### 이 sleep은 탭 내용 로딩 대기. 줄이거나 제거를 시도해볼 수 있음. 

            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")

            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)

                last_size = -1
                for _ in range(5):
                    if os.path.exists(downloaded_path):
                        current_size = os.path.getsize(downloaded_path)
                        if current_size > 0 and current_size == last_size:
                            break
                        last_size = current_size
                    time.sleep(1)

                df = pd.read_excel(downloaded_path, header=[0, 1, 2])

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = ['_'.join([str(c).strip() for c in col if str(c).strip() not in ('nan', '')]) for col in df.columns]
                else:
                    df.columns = [str(c).strip() for c in df.columns]
                
                df.columns = [col.strip() for col in df.columns]
                df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)

                print(f"[✅ 성공] {location_name} 데이터 수집 완료 (메모리 로드)")
                try:
                    os.remove(downloaded_path)
                except OSError as e:
                    print(f"다운로드된 파일 삭제 실패: {e}")
                return df
            else:
                print(f"[⚠️ 타임아웃] {location_name} (파일 다운로드 실패)")
                retries += 1
                print(f"[⚠️ 재시도] {location_name} - 다운로드 실패 (재시도 {retries}/2)")
                if len(driver.window_handles) > 1:
                    try:
                        driver.close()
                        driver.switch_to.window(main_window)
                    except Exception:
                        pass
                continue 

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{location_name}.png")
            time.sleep(0.5) ### 이 sleep은 에러 후 재시도 대기. 필요에 따라 줄이거나 제거 고려. 

        finally:
            if retries == 0 or retries == 2: 
                if len(driver.window_handles) > 1:
                    try:
                        driver.close()
                        driver.switch_to.window(main_window)
                    except Exception:
                        pass
    
    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")
    return None

# ---------------------- visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 + 내부 스크롤 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text, page_number):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")
    last_scroll_height = -1

    current_page_dfs = [] 

    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        if not buttons:
            print("버튼 없음 → 종료")
            break

        for idx, btn in enumerate(buttons):
            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()

                key = f"{branch_name}_{incident_count}_{idx}"
                
                # '합계' 항목 건너뛰기
                if branch_name == '합계':
                    print(f"🚫 건너뛰기: '{branch_name}' 항목은 처리하지 않습니다.")
                    visited.add(key) 
                    continue 

                if key in visited:
                    continue

                print(f"👉 클릭: {key}")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.2) ### 이 sleep은 스크롤 후 안정화 대기. 짧게 유지하거나 매우 신중하게 제거. 
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.5) ### 이 sleep은 팝업창 등장 대기. 줄이거나 제거를 시도해볼 수 있음. 

                df = handle_structure_download(driver, wait,
                                               f"{year_value}_{sido_text}_{jijace_text}_page{page_number}_{key}")
                if df is not None:
                    current_page_dfs.append(df)
                    visited.add(key)
                else:
                    visited.add(key) 
                
            except Exception as e:
                print(f"[❌ 클릭 실패] {e}")
                driver.save_screenshot(f"click_error_{key}.png") 
                visited.add(key) 

        current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
        max_scroll = driver.execute_script("return arguments[0].scrollHeight - arguments[0].clientHeight;", scroll_element)

        if current_scroll >= max_scroll:
            print("📌 스크롤 끝 → 종료")
            break
        
        if current_scroll == last_scroll_height: 
            print("스크롤 위치 변화 없음. 추가 콘텐츠 로딩 대기 시도. (3초)")
            time.sleep(0.5) ### 이 sleep은 콘텐츠 로딩 대기. 웹페이지 로딩 방식에 따라 조절 필요. 
            current_scroll_after_wait = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
            if current_scroll_after_wait == last_scroll_height:
                print("⛔️ 스크롤 위치 변화 없음 및 추가 대기 후에도 동일 → 현재 지자체 처리 강제 종료")
                break
        
        last_scroll_height = current_scroll

        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
        print("스크롤 다운...")
        time.sleep(0.5) ### 이 sleep은 스크롤 후 콘텐츠 로딩 대기. 줄이거나 제거를 시도해볼 수 있음. 

    print("✅ 이 페이지의 모든 버튼 다운로드 완료")
    return current_page_dfs

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    page = 1
    return download_page(driver, wait, year_value, sido_text, jijace_text, page) 
    # print("✅ 전체 다운로드 완료") # 이 print문은 download_page 반환 후 실행되지 않으므로 주석 처리 또는 위치 변경 필요

# ---------------------- 실행 ----------------------


test_years = ["2021", "2020"]

all_collected_dfs = [] 

for year_value in test_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
    # ✅ 인덱스 1 (서울)을 건너뛰고 인덱스 2부터 시작하도록 변경했습니다.
    for sido_index in range(1, len(sido_options)): 
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_index(sido_index)
        sido_text = sido_select.first_selected_option.text
        
        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        
        # 지자체 드롭다운을 '전체' (인덱스 0)로 고정
        jijace_select = Select(driver.find_element(By.ID, "jijace"))
        jijace_select.select_by_index(0) # 인덱스 0 = '전체' 선택
        jijace_text = jijace_select.first_selected_option.text # '전체' 텍스트 가져오기
        time.sleep(0.5) ### 이 sleep은 지자체 변경 후 데이터 로딩 대기. 제거를 시도해볼 수 있음. 

        search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_wrap > a")))
        driver.execute_script("arguments[0].click();", search_button)

        try:
            alert = driver.switch_to.alert
            alert.accept()
        except NoAlertPresentException:
            pass

        time.sleep(1) ### 이 sleep은 조회 결과 로딩 대기. WebDriverWait으로 대체 가능성을 고려해볼 것. 
        jijace_dfs = download_all(driver, wait, year_value, sido_text, jijace_text)
        if jijace_dfs:
            all_collected_dfs.extend(jijace_dfs)
        
        # ❌ 여기에 있던 불필요한 'break' 문을 제거했습니다.
        # 이제 모든 시도를 순회합니다.
    
    time.sleep(5) ### 이 sleep은 연도 전환 간 대기. 줄이거나 제거를 시도해볼 수 있음. 

driver.quit()

# ... (뒷부분 생략) ...


# --- 수집된 DF 확인 및 처리 ---

print("\n--- 스크래핑 후 데이터 처리 시작 ---")

if all_collected_dfs: # 리스트가 비어있지 않은지 확인
    print(f"총 수집된 DataFrame 개수: {len(all_collected_dfs)}")

    # 모든 DataFrame을 합치기
    final_df = pd.concat(all_collected_dfs, ignore_index=True)
    print("\n모든 DataFrame을 합친 최종 DataFrame의 상위 5행:")
    print(final_df.head())
    print(f"최종 DataFrame의 총 행 수 (합치기 직후): {len(final_df)}")
    
    # 중복 제거
    initial_rows = len(final_df)
    final_df.drop_duplicates(inplace=True)
    rows_after_dedup = len(final_df)
    print(f"중복 제거 전 행 수: {initial_rows}")
    print(f"중복 제거 후 행 수: {rows_after_dedup}")
    print(f"제거된 중복 행 수: {initial_rows - rows_after_dedup}")

    # 최종 DataFrame 파일로 저장 (예: 엑셀)
    output_excel_path = os.path.join(DOWNLOAD_DIR, "사고데이터_통합본.xlsx")
    final_df.to_excel(output_excel_path, index=False)
    print(f"\n✅ 최종 데이터가 '{output_excel_path}' 파일로 저장되었습니다.")

    # 최종 DataFrame 파일로 저장 (예: CSV - 필요시 주석 해제)
    # output_csv_path = os.path.join(DOWNLOAD_DIR, "사고데이터_통합본.csv")
    # final_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig') 
    # print(f"\n✅ 최종 데이터가 '{output_csv_path}' 파일로 저장되었습니다.")

else:
    print("\n수집된 DataFrame이 없어 최종 처리할 데이터가 없습니다.")

print("\n--- 스크래핑 및 데이터 처리 완료 ---")